In [127]:
import pandas as pd
import numpy as np

In [128]:
df = pd.read_excel("job-application-dataset.xlsx")
df.head(50)

,company_id,position_id,date_applied,location,industry,current_status,Unnamed: 6,platform,Unnamed: 8,Unnamed: 9,furthest_status
0,Anthem Blue Cross Blue Shield Nebraska,Data Intern,2023-09-19,NaN,NaN,Rejected,NaN,NaN,NaN,NaN,Rejected
1,Next Era Energy,Data Science Intern,2023-09-19,NaN,NaN,Rejected,NaN,NaN,NaN,NaN,Rejected
2,Wipfli,Data & Analytics Consulting Intern,2023-09-19,NaN,NaN,Rejected,NaN,NaN,NaN,NaN,Rejected
3,Dish,Data Analyst Intern,2023-09-19,NaN,NaN,Rejected,NaN,NaN,NaN,NaN,Rejected
4,Access Holdings,Summer Data Science Analyst Intern,2023-09-20,NaN,NaN,Waiting,NaN,NaN,NaN,NaN,Waiting
5,ACS,Data Science Intern,2023-09-20,NaN,NaN,Waiting,NaN,NaN,NaN,NaN,Waiting
6,Hussman,Data Analytics Intern,2023-09-20,NaN,NaN,Waiting,NaN,NaN,NaN,NaN,Waiting
7,Swire Coca-cola,Data Science Intern,2023-09-20,NaN,NaN,Waiting,NaN,NaN,NaN,NaN,Waiting
8,Converse,Data Engineering Intern,2023-09-20,NaN,NaN,Waiting,NaN,NaN,NaN,NaN,Waiting
9,MISO,Data Science/Data Analytics Intern,2023-09-20,NaN,NaN,Rejected,NaN,NaN,NaN,NaN,Rejected


Creating Companies Table

In [129]:
companies = pd.Series(df['company_id'].unique()).to_frame(name = "company_name").reset_index().copy()
companies = companies.rename(columns = {'index' : 'company_id'})
companies.to_csv("Companies.csv", index = False)

Creating Position Table

In [130]:
df['position_id']

0                             Data Intern
1                     Data Science Intern
2      Data & Analytics Consulting Intern
3                     Data Analyst Intern
4      Summer Data Science Analyst Intern
                      ...                
129           Medical data analyst intern
130                   Data analyst intern
131                   Data science intern
132                   Data science intern
133                  Data engineer intern
Name: position_id, Length: 134, dtype: object

Creating Status Table

In [131]:
df['current_status'].unique()

array(['Rejected', 'Waiting', 'Info session', 'Interview 1',
       'Final round', 'rejected'], dtype=object)

In [133]:
# convert all to lowercase
df['current_status'] = df['current_status'].str.lower()

# create table
status = pd.Series(df['current_status'].unique()).to_frame(name = "status").reset_index()
status = status.rename(columns = {'index' : 'status_id'})

# manually adding possible positions
extra_status = pd.DataFrame([[5, 'interview 2'], ['6', 'interview 3']], columns = ['status_id', 'status'])

status = pd.concat([status, extra_status], ignore_index=True)
status.to_csv('Status.csv', index = False)